In [ ]:
import os
import numpy as np

In [ ]:
test_file = 'test.1_all.hdf5_base.npz'
#test_file = 'dev.1_all.hdf5_base.npz'
test_path = './orig/omerlevy-bidaf_no_answer-2e9868b224e4/relation_splits/'

data = np.load( os.path.join(test_path,test_file) )
data.files  # ['predictions', 'targets']

In [ ]:
preds, targs = data['predictions'], data['targets']
preds.shape, targs.shape  # ((10, 1, 5, 128), (10, 1, 128))

In [ ]:
def assess(pred, targ):
    pass

idx=0; assess( preds[idx, 0, :, :], targs[idx, 0, :] )